In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import tensorflow as tf
import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)


from create_spectrogram_and_time_series_dataset_use import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

from SqueezeNet import SqueezeNet

In [7]:
sample_rate = 16000

In [8]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-18 18:56:09.856559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 18:56:09.857441: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-18 18:56:09.962782: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-18 18:56:09.969001: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [9]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [10]:
model = SqueezeNet(input_shape=(184, 80, 1), nb_classes=2)
model.summary()

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# EPOCHS = 1
EPOCHS = 10
history = model.fit(
    train_spectrogram_ds,
    epochs=EPOCHS
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 184, 80, 1)]         0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 92, 40, 96)           4800      ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling2D)     (None, 45, 19, 96)           0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv2D)      (None, 45, 19, 16)           1552      ['maxpool1[0][0]']            
                                                                                              

/home/polina/test/venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 108s 293ms/step - loss: 0.2523 - accuracy: 0.8957
Epoch 2/10
353/353 [==============================] - 98s 278ms/step - loss: 0.0958 - accuracy: 0.9710
Epoch 3/10
353/353 [==============================] - 109s 309ms/step - loss: 0.0844 - accuracy: 0.9758
Epoch 4/10
353/353 [==============================] - 83s 234ms/step - loss: 0.0643 - accuracy: 0.9827
Epoch 5/10
353/353 [==============================] - 82s 233ms/step - loss: 0.0440 - accuracy: 0.9872
Epoch 6/10
353/353 [==============================] - 79s 224ms/step - loss: 0.0338 - accuracy: 0.9910
Epoch 7/10
353/353 [==============================] - 80s 225ms/step - loss: 0.0512 - accuracy: 0.9875
Epoch 8/10
353/353 [==============================] - 81s 228ms/step - loss: 0.0270 - accuracy: 0.9931
Epoch 9/10
353/353 [==============================] - 79s 223ms/step - loss: 0.0296 - accuracy: 0.9919
Epoch 10/10
353/353 [==============================] - 78s 222ms/step - loss: 0.02

In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model, x_test_np, y_test_np, model_format="keras")


Validation dataset:
44/44 [==============================] - 4s 79ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.77%
Recall: 99.36%
Precision: 97.10%
F1-score: 98.22%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9821834731737121

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9809484989817016

Test dataset:
44/44 [==============================] - 3s 74ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9778618945971255

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9777210510822794


In [15]:
print("For Test dataset:")
evaluate_time_of_prediction(model, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)


For Test dataset:
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Time to make a prediction for a single data point
Mean: 0.078 seconds
Max: 0.412 seconds
Min: 0.051 seconds


In [16]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras"
keras.models.save_model(model, BASE_MODEL_FILE_NAME, save_format="keras")
print("Model file name: ", BASE_MODEL_FILE_NAME)
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "KB")

Model file name:  spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras
File size: 8687.448 Kilobytes


: 